In [1]:
from functions import preprossesing as prep
from functions import folding
import copy
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import LSTM, Input
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


filename = "running-example"
ocel, act_dict, cust_dict = prep.prepare_ocel(filename)
print(act_dict)
print(cust_dict)
drops_col_order = ["weight", "price", "Event_ID", 'Products']

ocel_orders = prep.flatten(ocel, 'Orders')
enriched_log, single_log = prep.gen_enriched_single_plus_csv(OCEL = ocel_orders,flatted_by = 'Orders',csvname = 'orders_complete', drops_col= drops_col_order)
np.random.seed(42)

{'place order': 'A', 'pick item': 'B', 'confirm order': 'C', 'item out of stock': 'D', 'reorder item': 'E', 'pay order': 'F', 'create package': 'G', 'send package': 'H', 'failed delivery': 'I', 'package delivered': 'J', 'payment reminder': 'K'}
{'Marco Pegoraro': 'a', 'Gyunam Park': 'b', 'Majid Rafiei': 'c', 'Junxiong Gao': 'd', 'Seran Uysal': 'e', 'Christina Rensinghof': 'f', 'Wil van der Aalst': 'g', 'Christine Dobbert': 'h', 'Luis Santos': 'i', 'Kefang Ding': 'j', 'Mohammadreza Fani Sani': 'k', 'Tobias Brockhoff': 'l', 'Anahita Farhang Ghahfarokhi': 'm', 'Mahnaz Qafari': 'n', 'Claudia Graf': 'o', 'Mahsa Bafrani': 'p', 'Lisa Mannel': 'q'}


In [2]:
filtered_index = ocel_orders.groupby('Packages')['Orders'] \
                            .apply(lambda x: len(x.unique())) \
                            #.loc[lambda x: x > 1].index
print(filtered_index)
## they are to many of them for one hot encodetbut i can use a flag if order is delvert with other orders in a package
filtered_index = ocel_orders.groupby('Customers')['Orders'] \
                            .apply(lambda x: len(x.unique())) \
                            #.loc[lambda x: x > 1].index
print(filtered_index)
## this seems a good columns to one hot encode 

Packages
          2000
660001       5
660002       2
660003       4
660004       4
          ... 
661321       1
661322       1
661323       1
661324       1
661325       1
Name: Orders, Length: 1326, dtype: int64
Customers
a    110
b    120
c    109
d    125
e    123
f    113
g    123
h    132
i    122
j    125
k    112
l    100
m    103
n    122
o    102
p    127
q    132
Name: Orders, dtype: int64


In [3]:
np.random.seed(42)

lastcase = ''
line = ''
firstLine = True
lines = []
Ptimeseqs = []
Ptimeseqs2 = []
Ptimeseqs3 = []
Ptimeseqs4 = []
nb_itemseqs = []

Ptimes = []
Ptimes2 = []
Ptimes3 = []
Ptimes4 = []
nb_items = []
ascii_offset = 64
Pcasestarttime = None
Plasteventtime = None

for index, row in enriched_log.iterrows():
    t = row['Timestamp']
    nb = row['Amount_Items']
    if row['Case_ID'] != lastcase:
        Stime = t
        Pcasestarttime = t
        Plasteventtime = t
        lastcase = row['Case_ID']
        if not firstLine:
            lines.append(line)
            Ptimeseqs.append(Ptimes)
            Ptimeseqs2.append(Ptimes2)
            Ptimeseqs3.append(Ptimes3)
            Ptimeseqs4.append(Ptimes4)
            nb_itemseqs.append(nb_items)

        line = ''
        Ptimes = []
        Ptimes2 = []
        Ptimes3 = []
        Ptimes4 = []
        nb_items = []

    line += row['Activity'] #unichr(int(row['Activity']) + ascii_offset)
    Ptimesincelastevent = t - Plasteventtime
    Ptimesincecasestart = t - Pcasestarttime
    midnight = t.replace(hour=0, minute=0, second=0, microsecond=0)
    timesincemidnight = t - midnight
    Ptimediff = 86400 * Ptimesincelastevent.days + Ptimesincelastevent.seconds
    Ptimediff2 = 86400 * Ptimesincecasestart.days + Ptimesincecasestart.seconds
    Ptimediff3 = timesincemidnight.seconds
    Ptimediff4 = t.weekday()
    Ptimes.append(Ptimediff)
    Ptimes2.append(Ptimediff2)
    Ptimes3.append(Ptimediff3)
    Ptimes4.append(Ptimediff4)
    nb_items.append(nb)
    Plasteventtime = t
    firstLine = False

# Add last case
lines.append(line)
Ptimeseqs.append(Ptimes)
Ptimeseqs2.append(Ptimes2)
Ptimeseqs3.append(Ptimes3)
Ptimeseqs4.append(Ptimes4)
nb_itemseqs.append(nb_items)

PtimeseqsF = []
for seq in Ptimeseqs2:
    PtimeseqsF.append([seq[-1] - t for t in seq])



divisor = np.mean([item for sublist in Ptimeseqs for item in sublist])  # average time between events
divisor2 = np.mean([item for sublist in Ptimeseqs2 for item in sublist])  # average time between current and first events
divisorTR = np.mean([item for sublist in PtimeseqsF for item in sublist])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print(f'numoof lines {len(lines)}')
print(len(nb_itemseqs))
### folding the lists 

divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932
numoof lines 2000
2000


In [6]:
# replaces lines activity sequences 
activity_sequences = enriched_log.groupby('Case_ID')['Activity'].apply(lambda x: ''.join(x)).reset_index()
# replaces Ptimeseqs (time difference between events) but care it is not grouped for the case_ID
enriched_log['Time_Diff'] = enriched_log.groupby('Case_ID')['Timestamp'].diff().fillna(pd.Timedelta(seconds=0)).dt.total_seconds().astype(int)
# replaces Ptimeseqs2 (times since the start of the case) but care it is not grouped for the case_ID
enriched_log['Time_Since_Start'] = (enriched_log['Timestamp'] - enriched_log.groupby('Case_ID')['Timestamp'].transform('first')).dt.total_seconds().astype(int)
# replaces Ptimeseqs3 (time since the midnight) but care it is not grouped for the case_ID
enriched_log['Time_Since_Midnight'] = (enriched_log['Timestamp'] - enriched_log['Timestamp'].dt.normalize()).dt.total_seconds().astype(int)
# replaces Ptimeseqs4 (just weekday) but care it is not grouped for the case_ID
enriched_log['Weekday'] = enriched_log['Timestamp'].dt.weekday
# Calculate the remaining time for each case but care it is not grouped for the case_ID
enriched_log['Remaining_Time'] = enriched_log.groupby('Case_ID')['Time_Since_Start'].transform(lambda x: x.max() - x)

from itertools import chain

# Check if activity sequences match with 'lines' column
activity_match = sum(activity_sequences['Activity'] != lines)
print(f"Activity sequences match 'lines' column: {activity_match} mismatch(es).")

# Check if Ptimeseqs match with 'Time_Diff' column
time_diff_match = sum(list(chain(*Ptimeseqs)) != enriched_log['Time_Diff'])
print(f"Ptimeseqs match 'Time_Diff' column: {time_diff_match} mismatch(es).")

# Check if Ptimeseqs2 match with 'Time_Since_Start' column
time_start_match = sum(list(chain(*Ptimeseqs2)) != enriched_log['Time_Since_Start'])
print(f"Ptimeseqs2 match 'Time_Since_Start' column: {time_start_match} mismatch(es).")

# Check if Ptimeseqs3 match with 'Time_Since_Midnight' column
time_midnight_match = sum(list(chain(*Ptimeseqs3)) != enriched_log['Time_Since_Midnight'])
print(f"Ptimeseqs3 match 'Time_Since_Midnight' column: {time_midnight_match} mismatch(es).")

# Check if Ptimeseqs4 match with 'Weekday' column
weekday_match = sum(list(chain(*Ptimeseqs4)) != enriched_log['Weekday'])
print(f"Ptimeseqs4 match 'Weekday' column: {weekday_match} mismatch(es).")

# Check if nb_itemseqs match with 'Amount_Items' column
nb_items_match = sum(list(chain(*nb_itemseqs)) != enriched_log['Amount_Items'])
print(f"nb_itemseqs match 'Amount_Items' column: {nb_items_match} mismatch(es).")

# Check if Ptimeseqs4 match with 'Remaining_Time' column
remaining_time_match = sum(list(chain(*PtimeseqsF)) != enriched_log['Remaining_Time'])
print(f"PtimeseqsF match 'Remaining_Time' column: {remaining_time_match} mismatch(es).")

T__divisor = np.mean([item for sublist in Ptimeseqs for item in sublist])  # average time between events
T__divisor2 = np.mean([item for sublist in Ptimeseqs2 for item in sublist])  # average time between current and first events
T__divisorTR = np.mean([item for sublist in PtimeseqsF for item in sublist])  # average time instance remaining
print(f"divisor: {T__divisor}")
print(f"divisor2: {T__divisor2}")
print(f"divisorTR: {T__divisorTR}")

divisor = np.mean(enriched_log['Time_Diff'])  # average time between events
divisor2 = np.mean(enriched_log['Time_Since_Start'])  # average time between current and first events
divisorTR = np.mean(enriched_log['Remaining_Time'])  # average time instance remaining
print(f"divisor: {divisor}")
print(f"divisor2: {divisor2}")
print(f"divisorTR: {divisorTR}")
print('The difference between the divisors are:')
print(T__divisor -divisor)
print(T__divisor2 -divisor2)
print(T__divisorTR -divisorTR)

print(f'numoof lines {len(lines)}')

Activity sequences match 'lines' column: 0 mismatch(es).
Ptimeseqs match 'Time_Diff' column: 0 mismatch(es).
Ptimeseqs2 match 'Time_Since_Start' column: 0 mismatch(es).
Ptimeseqs3 match 'Time_Since_Midnight' column: 0 mismatch(es).
Ptimeseqs4 match 'Weekday' column: 0 mismatch(es).
nb_itemseqs match 'Amount_Items' column: 0 mismatch(es).
PtimeseqsF match 'Remaining_Time' column: 0 mismatch(es).
divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932
divisor: 103436.38622985176
divisor2: 604261.1036767652
divisorTR: 1248093.355995932
The fierrence betwen the divisors are:
0.0
0.0
0.0
numoof lines 2000


In [7]:
enriched_log

,Case_ID,Activity,Timestamp,Items,Customers,Packages,Amount_Items,Time_Diff,Time_Since_Start,Time_Since_Midnight,Weekday,Remaining_Time
0,990001,A,2019-05-20 09:07:47,"[880001, 880004, 880003, 880002]",a,,4,0,0,32867,0,3197465
3,990001,C,2019-05-20 11:13:54,"[880001, 880004, 880003, 880002]",a,,4,7567,7567,40434,0,3189898
4,990001,B,2019-05-20 11:20:13,[880002],a,,1,379,7946,40813,0,3189519
7,990001,D,2019-05-20 13:54:37,[880004],a,,1,9264,17210,50077,0,3180255
25,990001,E,2019-05-21 10:03:49,[880004],a,,1,72552,89762,36229,1,3107703
...,...,...,...,...,...,...,...,...,...,...,...,...
32075,992000,B,2020-05-19 10:20:22,[888159],q,,1,70279,71336,37222,1,214226
32165,992000,G,2020-05-21 09:04:20,"[888014, 888159, 888146, 888144, 888145]",q,661303,5,168238,239574,32660,3,45988
32190,992000,H,2020-05-21 15:27:54,"[888014, 888159, 888146, 888144, 888145]",q,661303,5,23014,262588,55674,3,22974
32209,992000,J,2020-05-21 21:43:23,"[888014, 888159, 888146, 888144, 888145]",q,661303,5,22529,285117,78203,3,445


In [ ]:

lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5, lines_t6 = folding.folding_this(lines,Ptimeseqs,Ptimeseqs2,Ptimeseqs3,Ptimeseqs4,nb_itemseqs,PtimeseqsF,seeded = 42)

step = 1
sentences = []
softness = 0
next_chars = []
lines = list(map(lambda x: x + '!', lines))  # put delimiter symbol
maxlen = max(map(lambda x: len(x), lines))  # find maximum line size

# next lines here to get all possible characters for events and annotate them with numbers
# chars = map(lambda x: set(x),lines)
chars = list(set().union(*map(lambda x: set(x), lines)))
chars.sort()
target_chars = copy.copy(chars)
chars.remove('!')
print('total chars: {}, target chars: {}'.format(len(chars), len(target_chars)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
target_char_indices = dict((c, i) for i, c in enumerate(target_chars))
target_indices_char = dict((i, c) for i, c in enumerate(target_chars))
print(indices_char)

sentences_t = []
sentences_t2 = []
sentences_t3 = []
sentences_t4 = []
sentences_t5 = []
sentences_t6 = []


next_chars_t = []
next_chars_t2 = []
next_chars_t3 = []
next_chars_t4 = []
next_chars_t5 = []
next_chars_t6 = []


for line, line_t, line_t2, line_t3, line_t4, line_t5, line_t6 in zip(lines, lines_t, lines_t2, lines_t3, lines_t4, lines_t5, lines_t6):
    for i in range(0, len(line), step):
        if i == 0:
            continue

        # we add iteratively, first symbol of the line, then two first, three...

        sentences.append(line[0: i])
        sentences_t.append(line_t[0:i])
        sentences_t2.append(line_t2[0:i])
        sentences_t3.append(line_t3[0:i])
        sentences_t4.append(line_t4[0:i])
        sentences_t5.append(line_t5[0:i])
        sentences_t6.append(line_t6[0:i])
        next_chars.append(line[i])
        if i == len(line) - 1:  # special case to deal time of end character
            next_chars_t.append(0)
            next_chars_t2.append(0)
            next_chars_t3.append(0)
            next_chars_t4.append(0)
            next_chars_t5.append(0)
            next_chars_t6.append(0)


        else:
            next_chars_t.append(line_t[i])
            next_chars_t2.append(line_t2[i])
            next_chars_t3.append(line_t3[i])
            next_chars_t4.append(line_t4[i])
            next_chars_t5.append(line_t5[i])
            next_chars_t6.append(line_t6[i])

print('nb sequences:', len(sentences))

print('Vectorization...')
num_features = len(chars) + 6
print('num features: {}'.format(num_features))
X = np.zeros((len(sentences), maxlen, num_features), dtype=np.float32)
y_a = np.zeros((len(sentences), len(target_chars)), dtype=np.float32)
y_t = np.zeros((len(sentences)), dtype=np.float32)
y_tr = np.zeros((len(sentences)), dtype=np.float32)
for i, sentence in enumerate(sentences):
    leftpad = maxlen - len(sentence)
    next_t = next_chars_t[i]
    next_tr = next_chars_t6[i] # wrong in original code was t5

    sentence_t = sentences_t[i]
    sentence_t2 = sentences_t2[i]
    sentence_t3 = sentences_t3[i]
    sentence_t4 = sentences_t4[i]
    sentence_t5 = sentences_t5[i]

    for t, char in enumerate(sentence):
        for c in chars:
            if c == char:  # this will encode present events to the right places
                X[i, t + leftpad, char_indices[c]] = 1
        X[i, t + leftpad, len(chars)] = t + 1
        X[i, t + leftpad, len(chars) + 1] = sentence_t[t] / divisor
        X[i, t + leftpad, len(chars) + 2] = sentence_t2[t] / divisor2
        X[i, t + leftpad, len(chars) + 3] = sentence_t3[t] / 86400
        X[i, t + leftpad, len(chars) + 4] = sentence_t4[t] / 7

    for c in target_chars:
        if c == next_chars[i]:
            y_a[i, target_char_indices[c]] = 1 - softness
        else:
            y_a[i, target_char_indices[c]] = softness / (len(target_chars) - 1)
    y_t[i] = next_t / divisor
    y_tr[i] = next_tr / divisorTR

    np.set_printoptions(threshold=sys.maxsize)


In [ ]:
print(f"Shape of X: {X.shape}")
print(f"Shape of y_a: {y_a.shape}")
print(f"Shape of y_t: {y_t.shape}")
print(f"Shape of y_tr: {y_tr.shape}")
# Reshape the 3D array into 2D
reshaped_data = X.reshape((-1, X.shape[-1]))

# Convert reshaped ndarray to DataFrame
df = pd.DataFrame(reshaped_data)

# Print the resulting DataFrame
display(df[77:85])#.drop_duplicates())
display(df[40:42])#.drop_duplicates())

In [ ]:
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=True, dropout=0.2)(
    main_input)  # the shared layer
b1 = tf.keras.layers.BatchNormalization()(l1)
l2_1 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in activity prediction
b2_1 = tf.keras.layers.BatchNormalization()(l2_1)
l2_2 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time prediction
b2_2 = tf.keras.layers.BatchNormalization()(l2_2)
l2_3 = LSTM(100, implementation=2, kernel_initializer='glorot_uniform', return_sequences=False, dropout=0.2)(
    b1)  # the layer specialized in time remaining prediction
b2_3 = tf.keras.layers.BatchNormalization()(l2_3)
act_output = Dense(len(target_chars), activation='softmax', kernel_initializer='glorot_uniform', name='act_output')(
    b2_1)
time_output = Dense(1, kernel_initializer='glorot_uniform', name='time_output')(b2_2)

timeR_output = Dense(1, kernel_initializer='glorot_uniform', name='timeR_output')(b2_3)

model = Model(inputs=[main_input], outputs=[act_output, time_output, timeR_output])

model.compile(loss={'act_output': 'categorical_crossentropy', 'time_output': 'mae', 'timeR_output': 'mae'}, optimizer='nadam')

early_stopping = EarlyStopping(monitor='val_loss', patience=50)
model_checkpoint = ModelCheckpoint('./output_files/models/model_'+filename+'_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss',
                                   verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=15, verbose=0, mode='auto', min_delta=0.0001,
                               cooldown=0, min_lr=0)

history = model.fit(X, {'act_output': y_a, 'time_output': y_t, 'timeR_output': y_tr}, validation_split=0.2, verbose=2,
          callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, epochs=500)
# list all data in history

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
# plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()
# # summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()